In [6]:
import boto3 
bedrock = boto3.client(service_name='bedrock')

response = bedrock.get_foundation_model(modelIdentifier='anthropic.claude-3-5-sonnet-20241022-v2:0')

In [4]:
print(response['modelDetails']['modelArn'])

arn:aws:bedrock:us-west-2::foundation-model/anthropic.claude-3-5-sonnet-20241022-v2:0


In [ ]:
# Create a Bedrock agent runtime client. Potentially to use a different for KB:
aws_region = 'us-west-2'
bedrock_kb_client = boto3.client('bedrock-agent-runtime', region_name=aws_region)
bedrock_client = boto3.client(service_name='bedrock', region_name=aws_region)

In [8]:
# Get Model ARN from Model ID
model_id = "anthropic.claude-3-5-sonnet-20241022-v2:0"
response = bedrock_client.get_foundation_model(modelIdentifier=model_id)
model_arn = response['modelDetails']['modelArn']

In [40]:
user_query = "Tienes drones?"
temperature = 0.5
top_p = 0.5
knowledge_base_id = "1Y2HQZHIIY"
max_results = 3

In [41]:
# Query KB
response = bedrock_kb_client.retrieve_and_generate_stream(
    input={
        'text': user_query
    },
    retrieveAndGenerateConfiguration={
        'knowledgeBaseConfiguration': {
            'generationConfiguration': {
                'inferenceConfig': {
                    'textInferenceConfig': {
                        'maxTokens': 8192,
                        'temperature': temperature,
                        'topP': top_p
                    }
                },
            },
            'knowledgeBaseId': knowledge_base_id,
            'modelArn': model_arn,
            'retrievalConfiguration': {
                'vectorSearchConfiguration': {
                    'numberOfResults': max_results,
                    'overrideSearchType': 'HYBRID', # Or SEMANTIC
                }
            }
        },
        'type': 'KNOWLEDGE_BASE'
    }
)

In [47]:
# Looping through the response from the converse_stream api call
stream = response.get('stream')
streaming_text = ""
for event in stream:
    if 'output' in event:
        # Add the streaming chunks to our place holder
        streaming_text += event['output']['text']
print(streaming_text)
print(response)


I could not find any customer reviews specifically for the Syma X11 in the search results. While there is a mention of Syma X11 parts, there are no actual reviews of the drone itself.

{'ResponseMetadata': {'RequestId': '04a1ef20-3a8e-406e-9837-5730e26c31c5', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Tue, 10 Dec 2024 18:59:25 GMT', 'content-type': 'application/vnd.amazon.eventstream', 'transfer-encoding': 'chunked', 'connection': 'keep-alive', 'x-amzn-requestid': '04a1ef20-3a8e-406e-9837-5730e26c31c5', 'x-amzn-bedrock-knowledge-base-session-id': '1af8bd3d-a7b9-4362-bc35-369a1b74239f'}, 'RetryAttempts': 0}, 'sessionId': '1af8bd3d-a7b9-4362-bc35-369a1b74239f', 'stream': <botocore.eventstream.EventStream object at 0x10d3b1040>}


In [43]:
response

{'ResponseMetadata': {'RequestId': '987f3a47-cde4-455d-957f-e4830494a047',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 10 Dec 2024 18:57:29 GMT',
   'content-type': 'application/vnd.amazon.eventstream',
   'transfer-encoding': 'chunked',
   'connection': 'keep-alive',
   'x-amzn-requestid': '987f3a47-cde4-455d-957f-e4830494a047',
   'x-amzn-bedrock-knowledge-base-session-id': '1af8bd3d-a7b9-4362-bc35-369a1b74239f'},
  'RetryAttempts': 0},
 'sessionId': '1af8bd3d-a7b9-4362-bc35-369a1b74239f',
 'stream': <botocore.eventstream.EventStream at 0x10d43da90>}

In [44]:
new_user_query = "De esa lista que me das, ¿qué reviews tiene el primer producto?"

In [46]:
# Query KB
response = bedrock_kb_client.retrieve_and_generate_stream(
    input={
        'text': new_user_query
    },
    retrieveAndGenerateConfiguration={
        'knowledgeBaseConfiguration': {
            'generationConfiguration': {
                'inferenceConfig': {
                    'textInferenceConfig': {
                        'maxTokens': 8192,
                        'temperature': temperature,
                        'topP': top_p
                    }
                },
            },
            'knowledgeBaseId': knowledge_base_id,
            'modelArn': model_arn,
            'retrievalConfiguration': {
                'vectorSearchConfiguration': {
                    'numberOfResults': max_results,
                    'overrideSearchType': 'HYBRID', # Or SEMANTIC
                }
            }
        },
        'type': 'KNOWLEDGE_BASE'
    },
    sessionId=response['sessionId']
)